<a href="https://colab.research.google.com/github/anthonyhu25/Bayesian-Queuing-/blob/main/Bayesian_Queueing_Inference_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introduction


For the $M/G/1$ queueing model, customers arrive with independent interarrival times $W_{i} \sim Exp(\theta_{3}) = \theta_{3}e^{-\theta_{3}w_{i}}$

Customers have a service time $U_{i} \sim Uniform(\theta_{1}, \theta_{2})$ distribution. We only observe interdeparture times $Y_{i}$. Purpose of model is to infer the parameters $\theta = (\theta_{1}, \theta_{2}, \theta_{3})$ from the observed interdeparture times $y = (y_{1},...,y_{n})$.

We can represent the interdeparture times $Y_{i}$ as:

$\large Y_{i} = U_{i} + max(0, \sum_{j=1}^{i}W_{j} - \sum_{j=1}^{i-1}Y_{j}) = U_{i} + max(0, V_{i}-X_{i-1})$

where $X_{i} = \sum_{j=1}^{i}$ is the departure time of $i$-th customer, $V_{i}= \sum_{j=1}^{i}W_{j}$ is the arrival time of $i$-th customer, and $W_{j}$ difference in time between arrival times of $j$-th customer and $(j-1)$-th customer.

In the paper, they view the arrival times $V_{i}$ as latent(unobserved) variables, and summarized the conditional distributions with regards to previous arrival times and departure times. But, we need to sample $V_{i}$ conditional on the observed interdepature times $y$, and the previous $V_{i}$ samples (we denote this as $V_{-i}$)

## 2.1 Gibbs update for arrival time

To sample the $V_{i}$, we will use standard Gibbs sampling updates, which will need the conditional density of $V_{i}$ given other $V_{i}$-s and $\theta$.

There are 3 cases presented in the paper: when $i = 1$, $2 \le i \le n-1$, and $i = n$

Case 1: $i = 1$

$\large P(v_{1}|v_{-1}, y, θ) ∝ P(v_{1})P(y_{1}|v_{1},θ)P(v_{2}|v_{1},θ)$

$\large ∝ [\theta_{3}exp(-\theta_{3}v_{1})I(v_{1} \ge 0)]\frac{I(y_{1} ∈ [\theta_{1} + v_{1}, \theta_{2} + v_{1}])}{\theta_{2}-\theta_{1}}[\theta_{3}exp(-\theta_{3}(v_{2}-v_{1}))I(v_{1} \le v_{2})]$

$\large \propto I(v_{1} \in [max(0, x_{1} - \theta_{1}), min(v_{2}, x_{1} - \theta_{1})] \sim Uniform(max(0, x_{1} - \theta_{1}), min(v_{2}, x_{1} - \theta_{1}))$

Note that the last line comes from rewriting the inequalities $\theta_{1} + v_{1} \le y_{1} \le \theta_{2} + v_{1}$ in terms of $v_{1}$, and also taking into consideration of the bounds $I(v_{1} \ge 0)$ and $I(v_{1} \le v_{2})$

Case 2: $ 2 \le i \le n - 1$

Note that $V_{i} - X_{i-1}$ is the time between the arrival of the ${i}$-th customer, and the departure of the $(i-1)$-th customer. So we are sampling interdeparture time $Y_{i}$, conditional on the arrival $V_{i}$ and $X_{i-1}$, and the service time in one step for the uniform service distribution.

Intuitively speaking, we know the arrival times $V_{i-1}$ and $V_{i+1}$, as well as all of the interdeparture times $Y_{i}$ and parameters $\theta$. So we expres probability of $V_{i}$ in terms of the service distribution, shifted based on the location of $V_{i-1}$ and $V_{i+1}$.

$\large P(v_{i}|v_{-i}, y, θ) ∝ P(v_{i}|v_{i-1}, θ)P(y_{i}|x_{i-1}, v_{i}, \theta)P(v_{i+1}|v_{i}, θ)$

$\large \propto \theta_{3}exp(-\theta_{3}(v_{i}-v_{i-1}))I(v_{i-1} \le v_{i})[\frac{y_{i} \in [\theta_{1} + max(0, v_{i}-x_{i-1}), \theta_{2} + max(0, v_{i} - x_{i-1})]}{\theta_{2} - \theta_{1}}] \theta_{3}exp(-\theta_{3}(v_{i+1}-v_{i}))I(v_{i} \le v_{i+1})$

$\large \propto I(v_{i} \in [v_{i-1}, v_{i+1}])I(y_{i} \in [\theta_{1} + max(0, v_{i}-x_{i-1}), \theta_{2} + max(0, v_{i} - x_{i-1})])$

For this expression, we consider two cases: $Y_{i} > \theta_{2}$, and $Y_{i} \le \theta_{2}$, since we seek to establish the behavior of $Y_{i}$ conditional on the service distribution $U_{i}$, and as mentioned before $Y_{i} = U_{i} + max(0, V_{i} - X_{i-1})$

Since $U_{i} \sim Uniform(\theta_{1}, \theta_{2})$, $U_{i} \le \theta_{2}$ for all i. Hence, we examine $Y_{i}$ based on $\theta_{2}$.

If $Y_{i} > \theta_{2}$, this implies that $V_{i} > X_{i-1}$. So, $Y_{i} \sim Uniform(\theta_{1} + v_{i} -x_{i-1}, \theta_{2} + v_{i} - x_{i-1})$

